In [1]:
import sys
sys.path.append('../..')

import pandas as pd
from cryptolib.config import config
from cryptolib.exchange import Binance
from binance.client import Client

exchange = Binance(config.BINANCE_API_KEY, config.BINANCE_API_SECRET)

# Checking if the values are the same
print(exchange.get_last_price('ETHUSDT'))
print("sandbox:", exchange.sandbox)
print(config.BINANCE_API_KEY)

1811.65
sandbox: True
KA3SYnSVeaDull64B04HZiVtwGH313pTMMwwK7CLhozQNGWVdtntzRuZJ72199BR


## Orders

In [3]:
info = exchange.get_exchange_info()

In [ ]:
import pandas as pd

symbols = pd.DataFrame(info['symbols'])
symbols[symbols['symbol'].isin(['BTCUSDT'])].to_dict('records')

In [12]:
exchange.get_balance()

[{'asset': 'BNB', 'free': 1000.0, 'locked': 0.0},
 {'asset': 'BTC', 'free': 1.000547, 'locked': 0.0},
 {'asset': 'BUSD', 'free': 10000.0, 'locked': 0.0},
 {'asset': 'ETH', 'free': 100.0, 'locked': 0.0},
 {'asset': 'LTC', 'free': 500.0, 'locked': 0.0},
 {'asset': 'TRX', 'free': 499573.1, 'locked': 0.0},
 {'asset': 'USDT', 'free': 9967.86419167, 'locked': 0.0},
 {'asset': 'XRP', 'free': 49907.2, 'locked': 0.0}]

In [ ]:
exchange.get_klines('BTCUSDT', '1m', 10)

In [ ]:
exchange.create_order({
    'symbol': 'XRPUSDT',
    'side': 'SELL',
    'type': 'MARKET',
    'cost': '10000'
})

In [13]:
print(exchange.get_order('XRPUSDT', '1262854'))

{'info': {'symbol': 'XRPUSDT', 'orderId': '1262854', 'orderListId': '-1', 'clientOrderId': 'rL6PCK14YuAtvOpLMYJOsO', 'price': '0.00000000', 'origQty': '9969.10000000', 'executedQty': '9969.10000000', 'cummulativeQuoteQty': '4626.04979000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY', 'stopPrice': '0.00000000', 'icebergQty': '0.00000000', 'time': '1684572287260', 'updateTime': '1684572287260', 'isWorking': True, 'workingTime': '1684572287260', 'origQuoteOrderQty': '4626.08000000', 'selfTradePreventionMode': 'NONE'}, 'id': '1262854', 'clientOrderId': 'rL6PCK14YuAtvOpLMYJOsO', 'timestamp': 1684572287260, 'datetime': '2023-05-20T08:44:47.260Z', 'lastTradeTimestamp': None, 'symbol': 'XRP/USDT', 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 'reduceOnly': None, 'side': 'buy', 'price': 0.46403885907454034, 'triggerPrice': None, 'amount': 9969.1, 'cost': 4626.04979, 'average': 0.46403885907454034, 'filled': 9969.1, 'remaining': 0.0, 'status': 'closed',

In [13]:
import json
print(exchange.symbol_info['XRPUSDT']['NOTIONAL'])
print(exchange.symbol_info['XRPUSDT']['LOT_SIZE'])
print(exchange.symbol_info['XRPUSDT']['MARKET_LOT_SIZE'])

last_price = exchange.get_last_price('XRPUSDT')
print(exchange.get_max_order_amount('XRPUSDT'))
amount = (5000 / last_price)
cost = amount * last_price if amount < exchange.get_max_order_amount('XRPUSDT') else exchange.get_max_order_amount('XRPUSDT') * last_price * 0.997
print(amount)
print(cost)


{'filterType': 'NOTIONAL', 'minNotional': '10.00000000', 'applyMinToMarket': True, 'maxNotional': '9000000.00000000', 'applyMaxToMarket': False, 'avgPriceMins': 1}
{'filterType': 'LOT_SIZE', 'minQty': '0.10000000', 'maxQty': '90000.00000000', 'stepSize': '0.10000000'}
{'filterType': 'MARKET_LOT_SIZE', 'minQty': '0.00000000', 'maxQty': '10000.00000000', 'stepSize': '0.00000000'}
10000.0
10768.899418479432
4629.071


## Websockets

In [ ]:
from unicorn_binance_websocket_api.manager import BinanceWebSocketApiManager


def process_new_receives(stream_data):
    print(str(stream_data))


ubwa = BinanceWebSocketApiManager(exchange="binance.com")
ubwa.create_stream(channels=['trade', 'kline_1m'], markets=['btcusdt', 'bnbbtc', 'ethbtc'], 
                   process_stream_data=process_new_receives)

In [ ]:
ubwa.subscribe_to_stream(stream_id='c461aa945d90-d10c-b3c2-b9a7-2e0de8d6', channels=['kline_1m'], markets=['axsusdt', 'eduusdt'])

In [ ]:
ubwa.unsubscribe_from_stream(stream_id='c461aa945d90-d10c-b3c2-b9a7-2e0de8d6', markets=['btcusdt', 'ethbtc'])